# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import gradio as gr

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [15]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-

In [12]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [17]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [19]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/01/23/ll

In [6]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [40]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://www.campeones.com.ar/#/")
huggingface.links

['#content',
 '/tienda/',
 'https://campeones.com.ar/campeones-tv/',
 'https://twitter.com/Campeonesnet',
 'https://www.facebook.com/campeones.com.ar',
 'https://www.instagram.com/campeonesnet/',
 'https://campeones.com.ar/',
 'https://campeones.com.ar/ultimas-noticias-2/',
 'https://campeones.com.ar/category/nacionales/',
 'https://campeones.com.ar/category/internacionales/',
 'https://campeones.com.ar/calendarios/',
 'https://campeones.com.ar/campeonatos/',
 'https://campeones.com.ar/ultimas-noticias-2/',
 'https://campeones.com.ar/category/nacionales/',
 'https://campeones.com.ar/category/internacionales/',
 'https://campeones.com.ar/calendarios/',
 'https://campeones.com.ar/campeonatos/',
 'https://campeones.appradio.ar',
 'https://goo.su/QWGQ6',
 'https://campeones.com.ar/category/nacionales/tc/',
 'https://campeones.com.ar/category/internacionales/formula-1/',
 'https://campeones.com.ar/category/nacionales/tn/',
 'https://campeones.com.ar/category/nacionales/procar4000/',
 'https

In [23]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [7]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [41]:
print(get_all_details("https://www.campeones.com.ar/#/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://campeones.com.ar/campeones-desde-1963/'}, {'type': 'store page', 'url': 'https://campeones.com.ar/tienda/'}, {'type': 'campeones tv page', 'url': 'https://campeones.com.ar/campeones-tv/'}, {'type': 'news page', 'url': 'https://campeones.com.ar/ultimas-noticias-2/'}, {'type': 'company page', 'url': 'https://campeones.com.ar/campeonatos/'}]}
Landing page:
Webpage Title:
Portada | Campeones
Webpage Contents:
Ir al contenido
Últimas Noticias
Nacionales
Internacionales
CALENDARIOS
CAMPEONATOS
Últimas Noticias
Nacionales
Internacionales
CALENDARIOS
CAMPEONATOS
Buscar
Lo más leído:
TC
Fórmula 1
TN
Procar
MotoGP
Top Race
TC
Fórmula 1
TN
Procar
MotoGP
Top Race
ACTIVIDAD DE FIN DE SEMANA
Reflexión del campeón de TC y las carreras especiales: «No podemos ser casi 60 autos»
28 abril, 2025
Julián Santero considera que estas variantes podrían forzar sin necesidad al Turismo Carretera que es gustoso con sus carreras habituales.
TN: Ford, 

In [13]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [8]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [9]:
get_brochure_user_prompt("Campeones", "https://www.campeones.com.ar/#/")

Found links: {'links': [{'type': 'about page', 'url': 'https://campeones.com.ar/campeones-desde-1963/'}, {'type': 'company page', 'url': 'https://campeones.com.ar/campeones-tv/'}, {'type': 'company page', 'url': 'https://campeones.com.ar/campeones-radio/'}, {'type': 'news page', 'url': 'https://campeones.com.ar/ultimas-noticias-2/'}, {'type': 'calendars page', 'url': 'https://campeones.com.ar/calendarios/'}, {'type': 'championships page', 'url': 'https://campeones.com.ar/campeonatos/'}, {'type': 'careers page', 'url': 'https://campeones.com.ar/tienda/'}]}


'You are looking at a company called: Campeones\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nPortada | Campeones\nWebpage Contents:\nIr al contenido\nÚltimas Noticias\nNacionales\nInternacionales\nCALENDARIOS\nCAMPEONATOS\nÚltimas Noticias\nNacionales\nInternacionales\nCALENDARIOS\nCAMPEONATOS\nBuscar\nLo más leído:\nTC\nFórmula 1\nTC2000\nTurismo Pista\nTC\nFórmula 1\nTC2000\nTurismo Pista\nACTIVIDAD DE FIN DE SEMANA\nAnuncian que Colapinto correría en Imola con Alpine\n5 mayo, 2025\nDiversos medios destacaron en sus portales que el australiano fue informado que no seguirá en Fórmula 1 y que su puesto será ocupado por el argentino.\n¿Otro autódromo disponible para recibir categorías nacionales?\nAudi centraliza responsabilidades en el proyecto de Fórmula 1\nTurismo Pista: Se confirmó el circuito de la Carrera de los 300 Pilotos\n«Tenemos un gran auto pero 

In [10]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("Campeones", "https://www.campeones.com.ar/#/")

Found links: {'links': [{'type': 'about page', 'url': 'https://campeones.com.ar/campeones-desde-1963/'}, {'type': 'company page', 'url': 'https://campeones.com.ar/campeones-tv/'}, {'type': 'news page', 'url': 'https://campeones.com.ar/ultimas-noticias-2/'}, {'type': 'shop page', 'url': 'https://campeones.com.ar/tienda/'}, {'type': 'newsletter page', 'url': 'https://campeones.com.ar/revista-campeones/'}]}


# Campeones Brochure

## About Us

Welcome to **Campeones**, your premier destination for everything motorsport! We provide comprehensive coverage on various racing events including **Fórmula 1**, **Turismo Carretera (TC)**, **TC2000**, and **Turismo Pista**. Our dedication to delivering reliable news and insights makes us a trusted source for racing enthusiasts.

---

## Our Offerings

- **Latest News**: Stay updated with the latest happenings in the world of motorsport—both national and international. From updates on drivers to championship outcomes, we cover it all.
  
- **Race Calendars**: Check out the schedules of upcoming races to ensure you never miss a thrilling moment on the track.
  
- **Championships**: We provide in-depth analysis and coverage related to various championships, from the famous F1 to local racing circuits.

---

## Our Culture

At Campeones, we foster a culture of passion for racing and a commitment to community. Our team is made up of racing fans who thrive on delivering quality content while engaging with fellow enthusiasts. We believe in sportsmanship, teamwork, and the unifying spirit of motorsports, encouraging dialogue and knowledge sharing among our audience.

---

## Our Customers

Our audience includes motorsport fans, industry professionals, drivers, and teams who seek timely information and analysis on racing events. We serve those who are eager to learn, share, and celebrate a community built on high-speed thrills and competitive prowess.

---

## Careers at Campeones

We are always on the lookout for passionate individuals to join our team. Whether you're a seasoned journalist, a data analyst, or a marketing professional, Campeones offers a dynamic environment where you can excel and grow. Be part of a team dedicated to enhancing the motorsport experience for fans worldwide!

---

## Join Us!

Follow in the fast lane with Campeones! Stay connected with us on our website and on social media for the latest updates, in-depth articles, and community events that bring racing enthusiasts together.

For inquiries, potential partnerships, or career opportunities, please contact us through our website. We can't wait to share the excitement of motorsports with you! 

--- 

Thank you for considering **Campeones**—your ultimate motorsport companion!

In [20]:
gr.Interface(fn=create_brochure,
        inputs=[
            gr.Textbox(label="Company name:"),
            gr.Textbox(label="Landing page URL including http:// or https://")],
            outputs=[gr.Markdown(label="Brochure:")],      
            ).launch()

* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


Found links: {'links': [{'type': 'about page', 'url': 'https://campeones.com.ar/campeones-desde-1963/'}, {'type': 'company page', 'url': 'https://campeones.com.ar/campeones-tv/'}, {'type': 'careers page', 'url': 'https://campeones.com.ar/'}]}


# Campeones Company Brochure

---

## Welcome to Campeones

Campeones is your premier destination for all things motorsport. From national championships to international racing events including F1, TC, and Turismo Pista, we cover every thrilling moment on and off the track. Our mission is to keep fans informed and engaged with the latest news and updates in the world of motorsport.

### **Our Coverage**
- **Latest News**: Stay updated with our comprehensive coverage of local and international motorsport events.
- **Calendars and Championships**: Follow all upcoming races and championship standings, ensuring you never miss a moment of the action.
- **In-depth Analysis**: Get expert insights on race performances, team strategies, and driver dynamics.

---

## Culture at Campeones

At Campeones, we foster a dynamic and inclusive company culture that embraces teamwork, passion, and innovation. Our team consists of motorsport enthusiasts who are deeply committed to bringing the best content to our audience. We believe in collaboration and creativity, allowing our employees—whether in journalism, video production, or media management—to thrive and express their passion for motorsport. 

### **Values We Live By**
- **Passion for Motorsport**: Our love for racing drives everything we do.
- **Integrity and Accuracy**: We pride ourselves on delivering news and analyses that our audience can trust.
- **Community Engagement**: We actively engage with our viewers and fans, creating a community of like-minded motorsport lovers.

---

## Our Customers

Campeones is designed for both avid motorsport fans and casual viewers interested in the latest developments in racing. Our audience includes:

- **Motorsport Enthusiasts**: Individuals passionate about F1, TC, Turismo Pista, and more.
- **Teams and Drivers**: We provide valuable coverage that helps teams and individual athletes connect with their fans.
- **Companies and Sponsors**: Brands looking to engage with the motorsport community will find attractive partnership opportunities through our platform.

---

## Careers at Campeones

Join the exhilarating world of motorsport journalism and broadcasting at Campeones! We're always on the lookout for passionate individuals who want to contribute to our mission.

### **Current Opportunities**
- **Editorial Staff**: Become a part of our team to report on motorsport events, providing readers with the latest updates.
- **Content Creators**: Join our creative crew to produce engaging videos and social media content.
- **Marketing and Partnerships**: Help us grow our brand and enhance engagement with sponsors and fans.

If you have a passion for motorsport and want to be part of a growing company, we encourage you to explore career opportunities with us!

--- 

For more details on our services, culture, and career opportunities, visit us online at [Campeones](#) or reach out directly to our team.

Thank you for considering Campeones, where racing is not just a sport; it's a lifestyle!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [16]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )

    
    
    # response = ""
    # display_handle = display(Markdown(""), display_id=True)
    # for chunk in stream:
    #     response += chunk.choices[0].delta.content or ''
    #     response = response.replace("```","").replace("markdown", "")
    #     update_display(Markdown(response), display_id=display_handle.display_id)


    brochure = stream.choices[0].message.content
    system_propt_for_spanish= 'Sos un traductor experto en marketing, se te va pasar un folleto en ingles debés responder traducido en Espanol, la respuesta debe ser en markdown '

    translated_brochure = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_propt_for_spanish},
            {"role": "user", "content": brochure}
        ]
    )

    markdown_text = translated_brochure.choices[0].message.content

    # Aquí convertimos el markdown a HTML
    html_text = mark .markdown(markdown_text)

    # Mostramos el HTML en JupyterLab
    display(HTML(html_text))
    
    

In [17]:
stream_brochure("Campeones", "https://www.campeones.com.ar/#/")

Found links: {'links': [{'type': 'about page', 'url': 'https://campeones.com.ar/campeones-desde-1963/'}, {'type': 'company page', 'url': 'https://campeones.com.ar/'}, {'type': 'shop page', 'url': 'https://campeones.com.ar/tienda/'}, {'type': 'careers page', 'url': 'https://campeones.com.ar/campeones-tv/'}]}


NameError: name 'mark' is not defined

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("Campeones", "https://www.campeones.com.ar/#/")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>